In [7]:
import pandas as pd

In [8]:
def get_avg(Player,games=26,categories=None):
    stats = data.loc[data['Player']== Player]
    stats = stats.iloc[-games:,10:]
    avg = stats.mean()
    print(f'Jayson Tatum average stats over the last {games} games: ')
    return avg

def get_stat(Player,category,stats=pd.DataFrame([])):
    x = category.split('+')
    if stats.empty == True:
        stats = data.loc[data['Player']== Player]
    points = stats[x[0]]
    for i in range(len(x)-1):
        points = points + stats[x[i+1]]
    return points.to_list()


def get_line(player,pp_stat):
    guy = lines.loc[lines['player_name']== player]
    pt_lines = guy.loc[guy['stat']== pp_stat]
    return pt_lines

def moving_avg(Player,cat,window):
    
    #getting relevant stat category series
    stats = data.loc[data['Player']== Player]
    x = cat.split('+')
    total = stats[x[0]]
    
    #summing if category is multi stat
    for i in range(len(x)-1):
        total = total + stats[x[i+1]]
    total = np.array(total.to_list())

    #finding moving avg
    moving = [total[0]]
    for n in range(1,window-1):
        temp = total[0:n].mean()
        moving.append(temp)
    
    for i in range(len(total)-window+1):
        temp = total[i:i+window].mean()
        moving.append(temp)
        
    return moving

In [9]:
def print_prob(player,series,line,cat,games=None,prnt=True):
    
    p = 0
    n = 0
    if games == None:
        for i in series:
            n = n+1
            if i > line:
                p = p+1
        if prnt == True:
            print(f'{player} hits the {cat} line {p}/{n} times or',round(p/n,2),'%')
    else:
        p = 0
        n = 0
        for i in series[-games:]:
            n = n+1
            if i > line:
                p = p+1
        if prnt == True:
            print(f'{player} hits the {cat} line {p}/{n} times in his last 10') 
    return p/n
    

def graph_stat(player,cat,window):
    print(cat)
    mov = dynamic(player, cat ,window)
    points = np.array(get_stat(player,cat))
    line = get_line(player,cat)
    if line.empty == False:
        line = line.iloc[0,2]
        print('line:',line)
        print_prob(player,points,line,cat)
        print_prob(player,points,line,cat,10)
        plt.axhline(line, color='r', linestyle='--',label='line')
    
    x = range(len(points))
    plt.scatter(x,points, color='g',label=f'{cat}')#games
    plt.plot(x,points, color='g',linestyle=(0, (3, 6)))#game line
    plt.plot(x,mov,label='moving average')#moving average

    plt.axhline(np.mean(points), color='y', linestyle='-',label='season average')
    plt.ylabel(cat)
    plt.xlabel('Game #')
    plt.title(player)
    plt.legend()
    plt.show()
    print('avg:',round(np.mean(points),1))
    

In [10]:
def find_window(guy,cat):
    series,avg = get_stat(guy,cat)

    idx = []
    for window in range(2,10):

        pred = []
        mov = []
        for i in range(len(series)):
            mov = moving_avg(guy,cat,window)
        result = (((mov-series)**2).sum())/series.mean()
        idx.append(result)
    return idx
    


def dynamic(guy,cat,window=10):
    series = get_stat(guy,cat)
    series = np.array(series)
    alpha = 2
    
    pred = [series[0]]
    mov = [series[0]]
    for i in range(1,window):
            predict = series[0:i].mean()
            pred.append(predict)
            mov.append(predict)
    for i in range(window,len(series)):
        prev = mov[i-1]
        current = series[i]*(alpha/(window+1)) + mov[i-1]*(1- (alpha/(window+1)))
        #mov_pred = series[i-window:i].mean()
        mov.append(current)
    return(mov)


def find_dynamic(guy,cat):
    series,avg = get_stat(guy,cat)
    series = np.array(series)
    
    idx = []
    for window in range(2,10):
        mov = dynamic(guy,cat,window)
        result = (((mov-series)**2).sum())/series.mean()
        idx.append(result)
    return idx

def get_game(player,date):
    mask = (data['Date'] == date)
    df2 = data.loc[mask]
    df2 = df2.loc[data['Player']==player]
    return df2

In [1]:
def adjust(player,cat,opp):
    pos = get_pos(player)
    
    tm = opp_stats.loc[opp_stats['Team'] == opp]
    x = cat.split('+')
    adj = []
    final = 0
    for i in range(0,len(x)):
        opp_avg = position_adj(opp,x[i],pos)
        guy_avg = dynamic(player, x[i])
        res = opp_avg * guy_avg[-1]
        adj.append(res)
    for i in adj:
        final+= i
        
    
    return final

In [12]:
def best_odds(lines):
    player_lines = lines.iloc[:,0]

    vals = []
    for i in range(len(player_lines)):
        player = lines.iloc[i,0]
        if player in player_names:
            cat = lines.iloc[i,3]
            opp = lines.iloc[i,1]
            mov = dynamic(player, cat ,5)
            series = np.array(get_stat(player,cat))
            #line = get_line(player,cat)
            line = lines.iloc[i,2]
            #print(player,cat,line)
            season = print_prob(player,series,line,cat,prnt=False)
            l_10 = print_prob(player,series,line,cat,10,prnt=False)
            
            avg = np.mean(series)
            mov_avg = dynamic(player,cat)[-1]
            expected = adjust(player,cat,opp)
            vals.append([player,cat,round(avg,1),round(mov_avg,1),round(expected,1),line,l_10,season])
          
    odd = pd.DataFrame(vals,columns=['Player','Stat','Season_avg','mov_avg','expected','line','Last_10','Season'])
    odd['blend'] = odd[['Last_10', 'Season']].mean(axis=1)  
            
    return odd


In [13]:
def check(lines):
    val = []
    err = 0
    for i in range(len(lines)):
        player = lines['Player'][i]
        date = lines['Date'][i]
        cat = lines['Stat'][i]
        #cat = cat.split('+')
        line = lines['line'][i]
        game  = get_game(player,date)
        if game.empty:
            print('err')
            err += 1
        else:
            x = get_stat(player,cat,game)
            if x > line:
                val.append(1)
            else:
                val.append(0)
    return val


In [14]:
def get_pos(guy):
    return data.loc[data['Player']==guy]['Pos'].values[0]

In [15]:
def season_stats(player):
    stats = data.copy(deep=True)
    stats = stats.loc[stats['Player']==player]
    return stats[stat_columns].mean()

In [16]:
def position_diff(opp,cat):
    stats = data.copy(deep=True)
    stats = stats.loc[stats['Opp'] ==opp]
    for position in ['PG','SG','SF','PF','C']:
        frame = stats.loc[stats['Pos']==position]
        print(position,48*frame[cat].mean()/frame['MP'].mean())
    
    return frame.head(5)

In [ ]:
def position_adj(opp,cat,posit):
    stats = data.copy(deep=True)
    xs = []
    ys = []
    final = []
    stats2 = stats.loc[stats['Opp'] ==opp]
    pos = ['PG','SG','SF','PF','C']
    for position in pos:
        frame = stats.loc[stats['Pos']==position]
        x = 48*frame[cat].mean()/frame['MP'].mean()
        xs.append(x)
        frame2 = stats2.loc[stats2['Pos']==position]
        y = 48*frame2[cat].mean()/frame['MP'].mean()
        ys.append(y)
    for i in range(len(xs)):
        final.append((pos[i],ys[i]/xs[i]))
    for i in range(len(final)):
        if final[i][0] == posit:
            return final[i][1]


In [ ]:
def function_names():
    print('get_avg(Player,games=26,categories=None)')
    print('get_stat(Player,category,stats=pd.DataFrame([])):')
    print('get_line(player,pp_stat)')
    print('moving_avg(Player,cat,window)')
    print('print_prob(player,series,line,cat,games=None,prnt=True)')
    print('graph_stat(player,cat,window)')
    print('dynamic(guy,cat,window=10)')
    print('get_game(player,date)')
    print('adjust(player,cat,opp)')
    print('best_odds(lines)')
    print('check(lines)')
    print('get_pos(guy)')
    print('season_stats(player)')
    print('position_diff(opp,cat)')
    return
function_names()